In [1]:
import os
import sys
import json

# Use the current working directory instead of __file__
current_dir = os.getcwd()
parent_dir = os.path.abspath(os.path.join(current_dir, '..','parser'))

# Add the parent directory to sys.path
sys.path.append(parent_dir)

from antlr4 import *
from cobol85.Cobol85Lexer import Cobol85Lexer
from cobol85.Cobol85Parser import Cobol85Parser
from cobol85.Cobol85Listener import Cobol85Listener

print("Import Successful!")

Import Successful!


In [155]:
class CobolCFGListener(Cobol85Listener):

    def __init__(self):
        self.nodes = []
        self.edges = []
        self.business_rules = {}
        self.id = 0
        self.mem = 0

    def find_parent(self, current_ctx, target_ctx):
        """
        Utility function to recursively find specific type of parent context.
        """
        p = current_ctx.parentCtx
        if p is None:
            return None
        if isinstance(p,target_ctx):
            return p
        else:
            return self.find_parent(p, target_ctx)

    def find_child(self, ctx: ParserRuleContext, target_context_type: type) -> ParserRuleContext:
        """
        Utility function to recursively find the first child node of a given context type.
    
        :param ctx: The current parse tree node (context).
        :param target_context_type: The type of the target context to find.
        :return: The first child node of the target context type, or None if not found.
        """
        # Base case: if the current context is of the target type, return it
        if isinstance(ctx, target_context_type):
            return ctx
    
        # Recursively search in the children
        for child in ctx.children:
            if isinstance(child, ParserRuleContext):
                result = self.find_child(child, target_context_type)
                if result is not None:
                    return result
    
        # If no matching child is found, return None
        return None

    def enterProcedureDivision(self, ctx:Cobol85Parser.ProcedureDivisionContext):
        self.id +=1
        # Start processing procedure division for control flow
        self.nodes.append({"id": str(self.id), "data": {"label": "start"}})

    def enterAcceptStatement(self, ctx:Cobol85Parser.AcceptStatementContext):
        self.id +=1
        node = {"id": str(self.id), "data": {"label": f"ACCEPT {ctx.identifier().getText()}\n"}}
        self.nodes.append(node)
        self.edges.append({"id": str(self.id), "source": str(self.id-1), "target": str(self.id), "label": "sequential next"})

    def enterDivideStatement(self, ctx:Cobol85Parser.DivideStatementContext):
        self.id +=1
        # Start custom label
        label = "DIVIDE"
        # check identifier
        identifier = ctx.identifier()
        if identifier:
            label += " " + identifier.getText()
        else:
            label += " " + ctx.literal().getText()
        # check divideByGivingStatement
        divideByGivingStatement = ctx.divideByGivingStatement()
        if divideByGivingStatement:
            label += " " + "BY"
            identifier = divideByGivingStatement.identifier()
            if identifier:
                label += " " + identifier.getText()
            else:
                label += " " + divideByGivingStatement.literal().getText()
        # divideRemainder
        divideRemainder = ctx.divideRemainder()
        if divideRemainder:
            label += " " + "REMINDER" + " " + divideRemainder.identifier().getText()
        label += "\n"
        # End custom label
        node = {"id": str(self.id), "data": {"label": label}}
        self.nodes.append(node)
        self.edges.append({"id": str(self.id), "source": str(self.id-1), "target": str(self.id), "label": "sequential next"})

    def enterIfStatement(self, ctx: Cobol85Parser.IfStatementContext):
        self.id +=1
        self.mem = self.id
        node = {"id": str(self.id), "data": {"label": f"IF {ctx.condition().getText()}\n"}}
        self.nodes.append(node)
        self.edges.append({"id": str(self.id), "source": str(self.id-1), "target": str(self.id), "label": "sequential next"})
        # ifThen
        ifThen = ctx.ifThen()
        if ifThen:
            # statement
            statement = ifThen.statement()
            if statement:
                for i,st in enumerate(statement):
                    if i == 0:
                        self.id +=1
                        node = {"id": str(self.id), "data": {"label": f"{st.getText()}\n"}}
                        self.nodes.append(node)
                        self.edges.append({"id": str(self.id), "source": str(self.mem), "target": str(self.id), "label": "true"})
                    else:
                        self.id +=1
                        node = {"id": str(self.id), "data": {"label": f"{st.getText()}\n"}}
                        self.nodes.append(node)
                        self.edges.append({"id": str(self.id), "source": str(self.id-1), "target": str(self.id), "label": "sequential next"})
                        
        # ifElse
        ifElse = ctx.ifElse()
        if ifElse:
            # statement
            statement = ifElse.statement()
            if statement:
                for i,st in enumerate(statement):
                    if i == 0:
                        self.id +=1          
                        node = {"id": str(self.id), "data": {"label": f"{st.getText()}\n"}}
                        self.nodes.append(node)
                        self.edges.append({"id": str(self.id), "source": str(self.mem), "target": str(self.id), "label": "false"})
                    else:
                        self.id +=1
                        node = {"id": str(self.id), "data": {"label": f"{st.getText()}\n"}}
                        self.nodes.append(node)
                        self.edges.append({"id": str(self.id), "source": str(self.id-1), "target": str(self.id), "label": "sequential next"})               

In [156]:
input_code = """
IDENTIFICATION DIVISION.
PROGRAM-ID. EXAMPLE.
DATA DIVISION.
WORKING-STORAGE SECTION.
77  NUM         PICTURE 99.
77  QUOTIENT    PICTURE 99.
77  REMAIN      PICTURE 9.
PROCEDURE DIVISION.
   ACCEPT NUM.
   DIVIDE NUM BY 2 GIVING QUOTIENT REMAINDER REMAIN.
   IF REMAIN = 0
           DISPLAY NUM ' IS EVEN'
   ELSE
           DISPLAY  NUM ' IS ODD'
   END-IF.
   STOP RUN.
"""

lexer = Cobol85Lexer(InputStream(input_code))
stream = CommonTokenStream(lexer)
parser = Cobol85Parser(stream)

tree = parser.startRule()

In [157]:
cfg_listener = CobolCFGListener()
walker = ParseTreeWalker()
walker.walk(cfg_listener, tree)

In [158]:
data = {
    "nodes": cfg_listener.nodes,
    "edges": cfg_listener.edges,
}

with open('cfg.json', 'w') as f:
    json.dump(data, f)

In [159]:
data

{'nodes': [{'id': '1', 'data': {'label': 'start'}},
  {'id': '2', 'data': {'label': 'ACCEPT NUM\n'}},
  {'id': '3', 'data': {'label': 'DIVIDE NUM BY 2 REMINDER REMAIN\n'}},
  {'id': '4', 'data': {'label': 'IF REMAIN=0\n'}},
  {'id': '5', 'data': {'label': "DISPLAYNUM' IS EVEN'\n"}},
  {'id': '6', 'data': {'label': "DISPLAYNUM' IS ODD'\n"}}],
 'edges': [{'id': '2',
   'source': '1',
   'target': '2',
   'label': 'sequential next'},
  {'id': '3', 'source': '2', 'target': '3', 'label': 'sequential next'},
  {'id': '4', 'source': '3', 'target': '4', 'label': 'sequential next'},
  {'id': '5', 'source': '4', 'target': '5', 'label': 'true'},
  {'id': '6', 'source': '4', 'target': '6', 'label': 'false'}]}